## Deliverable 2. Create a Customer Travel Destinations Map.

In [52]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [53]:
# 1. Import the WeatherPy_database.csv file. 
# USed relative path and not direct path, worked to read to csv
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")

# Renaming "City Description" column to Current Description
city_data_df.rename(columns = {'City Description':'Current Description'}, inplace = True)
city_data_df.head()

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
0,0,Castro,-24.7911,-50.0119,58.41,63,7,10.89,BR,clear sky
1,1,Bredasdorp,-34.5322,20.0403,58.35,57,69,13.24,ZA,broken clouds
2,2,Plettenberg Bay,-34.0527,23.3716,54.12,87,100,11.99,ZA,moderate rain
3,3,Ushuaia,-54.8000,-68.3000,37.06,56,0,5.75,AR,clear sky
4,4,Mataura,-46.1927,168.8643,37.63,96,80,3.20,NZ,broken clouds


In [54]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for the trip? "))
max_temp = float(input("What is the maximum temperature you would like for the trip? "))

What is the minimum temperature you would like for the trip? 75
What is the maximum temperature you would like for the trip? 90


In [55]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                        (city_data_df["Max Temp"] >= min_temp)]

preferred_cities_df.head(10)

,City ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Current Description
7,7,Gamba,-2.6500,10.0000,75.13,72,38,8.88,GA,scattered clouds
8,8,Vila Franca Do Campo,37.7167,-25.4333,79.21,70,24,5.35,PT,few clouds
12,12,Puerto Ayora,-0.7393,-90.3518,75.16,94,100,7.00,EC,overcast clouds
16,16,Koindu,8.4386,-10.3253,80.58,71,93,6.46,GN,light rain
18,18,Hasaki,35.7333,140.8333,81.28,85,99,23.13,JP,overcast clouds
27,27,Ca Mau,9.1769,105.1500,79.27,85,96,3.06,VN,overcast clouds
33,33,Siemiatycze,52.4272,22.8623,81.23,48,99,11.07,PL,overcast clouds
34,34,Samarai,-10.6167,150.6667,78.37,86,100,11.81,PG,overcast clouds
38,38,Yokadouma,3.5167,15.0500,78.98,70,99,3.29,CM,overcast clouds
41,41,Altamira,-3.2033,-52.2064,84.29,89,75,6.91,BR,broken clouds


In [56]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City ID                298
City                   298
Lat                    298
Lng                    298
Max Temp               298
Humidity               298
Cloudiness             298
Wind Speed             298
Country                297
Current Description    298
dtype: int64

In [57]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.count()

City ID                297
City                   297
Lat                    297
Lng                    297
Max Temp               297
Humidity               297
Cloudiness             297
Wind Speed             297
Country                297
Current Description    297
dtype: int64

In [58]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
7,Gamba,GA,75.13,scattered clouds,-2.6500,10.0000,
8,Vila Franca Do Campo,PT,79.21,few clouds,37.7167,-25.4333,
12,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,
16,Koindu,GN,80.58,light rain,8.4386,-10.3253,
18,Hasaki,JP,81.28,overcast clouds,35.7333,140.8333,
27,Ca Mau,VN,79.27,overcast clouds,9.1769,105.1500,
33,Siemiatycze,PL,81.23,overcast clouds,52.4272,22.8623,
34,Samarai,PG,78.37,overcast clouds,-10.6167,150.6667,
38,Yokadouma,CM,78.98,overcast clouds,3.5167,15.0500,
41,Altamira,BR,84.29,broken clouds,-3.2033,-52.2064,


In [59]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]['name']
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [68]:
#clean_hotel_df = hotel_df.dropna()
import numpy as np
clean_hotel_df = hotel_df['Hotel Name'].replace('', np.nan, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Vila Franca Do Campo,PT,79.21,few clouds,37.7167,-25.4333,Pestana Bahia Praia
12,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
18,Hasaki,JP,81.28,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel
27,Ca Mau,VN,79.27,overcast clouds,9.1769,105.1500,Mường Thanh Luxury Cà Mau Hotel
33,Siemiatycze,PL,81.23,overcast clouds,52.4272,22.8623,Hotel Kresowiak Siemiatycze
...,...,...,...,...,...,...,...
672,Machaneng,BW,82.18,clear sky,-23.1862,27.4886,Matswelo Hotel
673,Toccoa,US,81.68,scattered clouds,34.5773,-83.3324,Quality Inn
674,Tarakan,ID,82.76,scattered clouds,3.3000,117.6333,RedDoorz Syariah near Taman Berlabuh Tarakan
675,Maceio,BR,85.44,broken clouds,-9.6658,-35.7353,Hotel Ponta Verde Maceió


In [69]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
8,Vila Franca Do Campo,PT,79.21,few clouds,37.7167,-25.4333,Pestana Bahia Praia
12,Puerto Ayora,EC,75.16,overcast clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
18,Hasaki,JP,81.28,overcast clouds,35.7333,140.8333,Inubosaki Kanko Hotel
27,Ca Mau,VN,79.27,overcast clouds,9.1769,105.1500,Mường Thanh Luxury Cà Mau Hotel
33,Siemiatycze,PL,81.23,overcast clouds,52.4272,22.8623,Hotel Kresowiak Siemiatycze
...,...,...,...,...,...,...,...
672,Machaneng,BW,82.18,clear sky,-23.1862,27.4886,Matswelo Hotel
673,Toccoa,US,81.68,scattered clouds,34.5773,-83.3324,Quality Inn
674,Tarakan,ID,82.76,scattered clouds,3.3000,117.6333,RedDoorz Syariah near Taman Berlabuh Tarakan
675,Maceio,BR,85.44,broken clouds,-9.6658,-35.7353,Hotel Ponta Verde Maceió


In [70]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [17]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))